$\newcommand{\vect}[1]{\vec{#1}}
\newcommand{\uvect}[1]{\hat{#1}}
\newcommand{\abs}[1]{\lvert#1\rvert}
\newcommand{\norm}[1]{\lVert#1\rVert}
\newcommand{\I}{\mathrm{i}}
\newcommand{\ket}[1]{\left|#1\right\rangle}
\newcommand{\bra}[1]{\left\langle#1\right|}
\newcommand{\braket}[1]{\langle#1\rangle}
\newcommand{\op}[1]{\mathbf{#1}}
\newcommand{\mat}[1]{\mathbf{#1}}
\newcommand{\d}{\mathrm{d}}
\newcommand{\pdiff}[3][]{\frac{\partial^{#1} #2}{\partial {#3}^{#1}}}
\newcommand{\diff}[3][]{\frac{\d^{#1} #2}{\d {#3}^{#1}}}
\newcommand{\ddiff}[3][]{\frac{\delta^{#1} #2}{\delta {#3}^{#1}}}
\DeclareMathOperator{\erf}{erf}
\DeclareMathOperator{\order}{O}
\DeclareMathOperator{\diag}{diag}$

#Random Non-linear Schrödinger Equation

Here is a small test-problem evolving a Schrödinger-like equation of the form

$$
  \I\ket{\dot{\psi}} = \mat{H}\ket{\psi}
$$

where the Hamiltonian depends non-linearly on $\ket{\psi}$.  We apply to this equation an external potential.

In [ ]:
from __future__ import division
import sys;sys.path.append('../../')

import numpy as np

from pytimeode import interfaces
from pytimeode.utils import interface

class Problem(object):
    def __init__(self, N=5, seed=1, g=1.0):
        np.random.seed(seed)
        H = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        H = H + H.conj().T
        
        V = np.random.rand(N, N) + np.random.rand(N, N)*1j - 0.5 - 0.5j
        V = V + V.conj().T
        
        Es, Psi = np.linalg.eigh(H)
        H -= Es[0]*np.eye(N)
        Es -= Es[0]
        psi0 = Psi[:, 0]
        H -= g*np.diag(abs(psi0)**2)
        
        self.H = H
        self.V = V
        self.g = g
        self.N = N

    def Vt(self, t):
        return 0.0*self.V
    
    def get_Hy(self, y, t):
        V = self.Vt(t) + self.g*abs(y)**2
        return (self.H + V).dot(y)

    
class State(interfaces.ArrayStateMixin):
    interface.implements(interfaces.IStateForABMEvolvers)
    
    def __init__(self, p):
        self.t = 0.0
        self.p = p
        self.data = np.empty(p.N, dtype=complex)
    
    def copy(self):
        return interfaces.ArrayStateMixin.copy(self, y=State(p=self.p))
    
    def copy_from(self, y):
        interfaces.ArrayStateMixin.copy_from(self, y)
        self.p = y.p
        
    def compute_dy(self, t, dy=None, potentials=None):
        if dy is None:
            dy = self.copy()
        dy.data[...] = self.p.get_Hy(y=self.data, t=t)
        dy *= -1j
        return dy
        
interface.verifyClass(interfaces.IStateForABMEvolvers, State)

In [ ]:
p = Problem()
y = State(p)
y.data[...] = 1.0
from pytimeode import evolvers
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(200)
with y.lock:
    evolver = evolvers.EvolverABM(y=y, dt=0.01)
    evolver.evolve(2)
evolver.y

#Gross-Pitaevskii Equation (GPE) 

##1D 

In [ ]:
from __future__ import division
import sys;sys.path.append('../../')
import numpy as np
from pytimeode import interfaces
from pytimeode.utils import interface

Here we briefly demonstrate using the framework to solve the GPE:

$$
  \I\hbar\pdiff{}{t}\psi(x,t) 
  = \left(\frac{-\hbar^2 \nabla^2}{2m} + g\abs{\psi(x,t)}^2 + V(x,t)\right)\psi(x,t).
$$

The state here is simply a complex array representing the wavefunction $\psi(x,t)$ so we can use the ``ArrayStateMixin``.  We represent the problem on an $N$-point lattice in a periodic box of length $L$ so we can use fourier methods.

As a test problem, we will look at a domain wall oscillating in a harmonic trap

$$
  V(x, t) = \frac{m\omega^2}{2}x^2 - \mu.
$$

The length-scales for this problem are the trap-length $a = \sqrt{\hbar/m\omega}$ and the inter-particle separation $1/\abs{\psi}^2$ and the interaction length-scale $l_g = \hbar^2/mg$.  If we assume that the Thomas-Fermi approximation is valid in the center of the trap, then the central density $n_0 = \mu/g$ is set by the chemical potential.  We choose units so that $\hbar = m = \omega = 1$, set the TF radius $R_{TF} = 8a$ so that $\mu = m\omega^2 R_{TF}^2/2 = 32 m\omega^2 a^2$.  Finally, we choose $g=\hbar^2/ma$ so that $l_g = a$.

In [ ]:
fft = np.fft.fft
ifft = np.fft.ifft

class State(interfaces.ArrayStateMixin):
    interface.implements(interfaces.IStateForABMEvolvers)
    
    def __init__(self, N=256, L=20.0, mu=32.0, hbar=1.0, m=1.0, g=1.0, omega=1.0):
        self.t = 0.0
        self.N = N
        self.L = L
        self.hbar = hbar
        self.m = m
        self.g = g
        self.mu = mu
        self.omega = omega
        
        self.data = np.empty(self.N, dtype=complex)
        self.dx = self.L / self.N
        
        # Abscissa centered about zero
        self.x = np.arange(N)*self.dx - L/2.0
        
        # Momenta
        self.k = 2*np.pi * np.fft.fftfreq(self.N, d=self.dx)
        self.k2 = self.k**2
        
    def braket(self, a, b):
        return a.conj().dot(b)*self.dx
    
    def n_TF(self):
        """Return the TF density."""
        V = self.Vt(t=0)
        n_TF = np.where(self.mu > V, 
                        (self.mu - V)/self.g,
                        0.0)
        return n_TF

    def laplacian(self, psi):
        """Return the laplacian of `f(x)`."""
        return - ifft(self.k2 * fft(psi))
    
    def Vt(self, t):
        """Overload this to define a time-dependent external potential."""
        return self.m/2.0 * (self.omega*self.x)**2
    
    def get_V(self, psi, t=0.0):
        """Return the effective potential."""
        return self.g * abs(psi)**2 - self.mu + self.Vt(t=t)
        
    def apply_H(self, psi, t=0.0):
        hbar2m = self.hbar**2/2.0/self.m
        V = self.get_V(psi=psi, t=t)
        return -hbar2m * self.laplacian(psi) + V * psi
        
    def compute_dy(self, t, dy=None, potentials=None):
        if dy is None:
            dy = self.copy()
        y = self.data.copy()
        Hy = self.apply_H(psi=self.data, t=t)
        Hy += self.braket(y, Hy)/self.braket(y,y) * y
        dy.data[...] = Hy
        dy *= -1j
        return dy
        
interface.verifyClass(interfaces.IStateForABMEvolvers, State)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
y0 = State()
y0.data[...] = np.sqrt(y0.n_TF())
plt.plot(y0.x, abs(y0.data)**2)

In [ ]:
from pytimeode import evolvers
with y0.lock:
    evolver = evolvers.EvolverABM(y=y0, dt=0.0005)
    evolver.evolve(3000)
plt.plot(y0.x, abs(evolver.y.data)**2)

In [ ]:
%qtconsole